In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Implementing a resizable generic list

One limitation of our generic list implementation is that our lists have a fixed capacity of 16. Can we implement a resizable list?

A careful review of the `FixedArrayList` implementation shows that the only method that depends on the length of the array is `add`:

```java
	/**
	 * Adds the specified element to the end of this list.
	 * 
	 * @param elem the element to add
	 * @throws RuntimeException if the list is full
	 */
	public void add(E elem) {
		if (this.size() == this.arr.length) {
			throw new RuntimeException("list is full");
		}
		// adjust back
		this.back++;
		this.arr[this.back] = elem;
	}
```

In `add` if the size of the list is equal to the length of the array then we know that the array is full and no more elements can be added to the list. Java arrays are not resizable so if we want to add more capacity to the list we need to:

1. make an array that is longer than `this.arr`
2. copy the elements of `this.arr` into the longer array
3. re-assign `this.arr` to refer to the longer array

We could do all of this manually:

```java
	public void add(E elem) {
		if (this.size() == this.arr.length) {
			E[] bigger = (E[]) (new Object[2 * this.size()]);
			for (int i = 0; i < this.size(); i++) {
				bigger[i] = this.arr[i];
			}
			this.arr = bigger;
		}
		// adjust back
		this.back++;
		this.arr[this.back] = elem;
	}
```

But there is a much easier way: Use the method `Arrays.copyOf`:

```java
	public void add(E elem) {
		if (this.size() == this.arr.length) {
			E[] bigger = Arrays.copyOf(this.arr, 2 * this.size());
			this.arr = bigger;
		}
		// adjust back
		this.back++;
		this.arr[this.back] = elem;
	}
```

Notice that we make the new array twice as long as the old array.

The complete implementation of `ResizableArrayList` is shown in the next cell. Run the cell to compile the class and then run the following cell to run the `main` method.

In [ ]:
package ca.queensu.cs.cisc235.list;

import java.util.Arrays;
import java.util.Objects;

/**
 * A minimal implementation of a fixed-size list of elements backed by an array.
 *
 * @param <E> the element type of this list
 */
public class ResizableArrayList<E> {

    /**
     * The array of strings in the list.
     */
    private E[] arr;
    
    /**
     * Index of the last element in the list; equal to -1 for an empty list.
     */
    private int back;      
    
    /**
     * Initializes this list to an empty list of capacity 16.
     */
    public ResizableArrayList() {
        this.arr = (E[]) (new Object[16]);
        this.back = -1;
    }
    
    /**
     * Returns the number of elements in this list.
     * 
     * @return the number of elements in this list
     */
    public int size() {
        return this.back + 1;
    }
    
    /**
     * Test if an index is valid for this list.
     * @param index
     */
    private void checkIndex(int index) {
        if (index < 0 || index >= this.size()) {
            throw new IndexOutOfBoundsException(index + " out of bounds");
        }
    }
    
    /**
     * Returns the element at the specified index of this list.
     * 
     * @param index the index of the element to get
     * @return the element at the specified index of this list
     * @throws IndexOutOfBoundsException if index is out of bounds
     */
    public E get(int index) {
        this.checkIndex(index);
        return this.arr[index];
    }
    
    /**
     * Sets the element at the specified index to the specified element in this
     * list returning the old element.
     *   
     * @param index the index of the element to set
     * @param elem the new element
     * @return the old element
     * @throws IndexOutOfBoundsException if index is out of bounds
     */
    public E set(int index, E elem) {
        this.checkIndex(index);
        E old = this.arr[index];
        this.arr[index] = elem;
        return old;
    }
    
    /**
     * Adds the specified element to the end of this list.
     * 
     * @param elem the element to add
     */
    public void add(E elem) {
        if (this.size() == this.arr.length) {
            E[] bigger = Arrays.copyOf(this.arr, 2 * this.size());
            this.arr = bigger;
        }
        // adjust back
        this.back++;
        this.arr[this.back] = elem;
    }
    
    /**
     * Removes the element at the specified index of this list returning 
     * the element that was removed. All elements after the specified index
     * are shifted one position to the front of the list.
     * 
     * @param index the index of the element to remove
     * @return the removed element
     * @throws IndexOutOfBoundsException if index is out of bounds
     */
    public E remove(int index) {
        this.checkIndex(index);
        E old = this.arr[index];
        
        // shift elements from (index + 1) down one index
        for (int i = index + 1; i <= this.back; i++) {
            this.arr[i - 1] = this.arr[i]; 
        }
        
        // null out old back element, otherwise it can't be garbage collected
        this.arr[this.back] = null;
        
        // adjust back
        this.back--;
        
        return old;
    }

    public static void main(String[] args) {
        ResizableArrayList<String> t = new ResizableArrayList<>();
        for (char c = 'a'; c < 'a' + 26; c++) {
            t.add("" + c);
            t.add("" + c);
        }
        System.out.println(Arrays.toString(t.arr));
        
        for (int i = 0; i < t.size(); i++) {
            System.out.println(t.get(i));
        }
        
        for (int i = 0; i < t.size(); i++) {
            String s = t.get(i);
            t.set(i, s + s);
        }
        System.out.println(Arrays.toString(t.arr));
        
        for (int i = 0; i < 52; i++) {
            t.remove(0);
            System.out.println(Arrays.toString(t.arr));
        }
        
        // force an exception
        t.set(0, "");
    }
}


In [ ]:
ca.queensu.cs.cisc235.list.ResizableArrayList.main(null);

## Exercises

1. What is the worst-case computational complexity of the `add` method for our list implementation?

2. Can you informally reason about the average complexity of the `add` method for our list implementation?

3. Besides computational complexity, what disadvantages do you see with our list implementation?

4. Instead of doubling the length of the array when we need to increase its capacity, what tradeoffs and benefits can you see if we made the array grow by a fixed number of elements (say we increased the length of the array by 16 elements whenever we need extra capacity).